# Cleaning California Data From USA.com

In [1]:
import json
import pandas as pd
import numpy as np
import glob
import csv
from faker import Faker
from faker.providers import internet
import datetime
from sqlalchemy import create_engine
import psycopg2
from config import db_password
import time

In [2]:
# path = r'../Resources1'
# filenames = glob.glob(path + '/*.csv')
# dfs = []
# for filename in filenames:
#     df=pd.read_csv(filename, index_col='zip', header=0)
#     dfs.append(df)
    
# usa_df = pd.concat(dfs, axis= 0, ignore_index=True)
# usa_df.drop('Unamed: 0', axis=1)
# usa_df.head()

## Average Education Index

In [24]:
avg_edu_2000 = pd.read_csv('../Resources1/avg_edu_index_2000.csv')
avg_edu_2000 = avg_edu_2000.drop('Unnamed: 0', axis=1)
avg_edu_2000.set_index('zip', inplace=True)
avg_edu_2000 = avg_edu_2000.astype(float)
avg_edu_2000.head()

Average Education Index 2000
zip                                
94305                         17.76
95041                         17.12
94708                         17.01
94304                         17.00
94707                         16.83

In [25]:
avg_edu_2014 = pd.read_csv('../Resources1/avg_edu_index_2014.csv')
avg_edu_2014 = avg_edu_2014.drop('Unnamed: 0', axis=1)
avg_edu_2014.set_index('zip', inplace=True)
avg_edu_2014 = avg_edu_2014.astype(float)
avg_edu_2014.head()

Average Education Index 2014
zip                                
95463                         19.00
95721                         19.00
94305                         17.69
95736                         17.68
92617                         17.38

In [57]:
avg_edu = pd.concat([avg_edu_2000,avg_edu_2014], axis=1, join='inner')
avg_edu['Index Change'] = avg_edu['Average Education Index 2014'] - avg_edu['Average Education Index 2000']
avg_edu.head()

Average Education Index 2000  Average Education Index 2014  \
zip                                                                 
94305                         17.76                         17.69   
95041                         17.12                         15.65   
94708                         17.01                         17.12   
94304                         17.00                         17.02   
94707                         16.83                         17.15   

       Index Change  
zip                  
94305         -0.07  
95041         -1.47  
94708          0.11  
94304          0.02  
94707          0.32

## Median Income

In [34]:
med_inc_2000 = pd.read_csv('../Resources1/median_income_2000.csv')
med_inc_2000 = med_inc_2000.drop('Unnamed: 0', axis=1)
med_inc_2000.set_index('zip', inplace=True)
med_inc_2000['Median Income 2000'] = med_inc_2000['Median Income 2000'].str.replace(',', '').str.replace('$','').astype(float)
med_inc_2000.head()

Median Income 2000
zip                      
94027            200001.0
92067            196298.0
94028            164479.0
94022            145425.0
94506            142459.0

In [35]:
med_inc_2014 = pd.read_csv('../Resources1/median_income_2014.csv')
med_inc_2014= med_inc_2014.rename(columns = {'Zip':'zip'})
med_inc_2014.set_index('zip', inplace=True)
med_inc_2014=med_inc_2014.drop('Unnamed: 0', axis=1)
med_inc_2014['Median_Household_Income_2014'] = med_inc_2014['Median_Household_Income_2014'].str.replace(',', '').str.replace('$','').astype(float)
med_inc_2014.head()

Median_Household_Income_2014
zip                                
94027                      236912.0
92145                      228587.0
91980                      200325.0
94957                      187857.0
94022                      182750.0

In [58]:
med_inc = pd.concat([med_inc_2000,med_inc_2014], axis=1, join='inner')
med_inc['Inflated Median Income 2000'] = 1.3993019 * med_inc['Median Income 2000']
med_inc['Income Change'] = med_inc['Median_Household_Income_2014'] - med_inc['Inflated Median Income 2000']
med_inc = med_inc.round(2)
med_inc.columns = ['Median Income 2000','Median Income 2014','Inflated Median Income 2000','Income Change']
med_inc = med_inc[['Median Income 2000','Inflated Median Income 2000','Median Income 2014','Income Change']]
med_inc.head()

Median Income 2000  Inflated Median Income 2000  Median Income 2014  \
zip                                                                          
94027            200001.0                    279861.78            236912.0   
92067            196298.0                    274680.16            119939.0   
94028            164479.0                    230155.78            180174.0   
94022            145425.0                    203493.48            182750.0   
94506            142459.0                    199343.15            176241.0   

       Income Change  
zip                   
94027      -42949.78  
92067     -154741.16  
94028      -49981.78  
94022      -20743.48  
94506      -23102.15

## Median Rent

In [38]:
med_rent_2000 = pd.read_csv('../Resources1/median_rent_2000.csv')
med_rent_2000.set_index('zip', inplace=True)
med_rent_2000 = med_rent_2000.drop('Unnamed: 0', axis=1)
med_rent_2000['Median_Rent_2000'] = med_rent_2000['Median_Rent_2000'].str.replace(',', '').str.replace('$','').astype(float)
med_rent_2000.head()

Median_Rent_2000
zip                    
92067            2001.0
92091            2001.0
92602            2001.0
93953            2001.0
94027            2001.0

In [39]:
med_rent_2014 = pd.read_csv('../Resources1/median_rent_2014.csv')
med_rent_2014.set_index('zip', inplace=True)
med_rent_2014 = med_rent_2014.drop('Unnamed: 0', axis=1)
med_rent_2014['Median_Rent_2014'] = med_rent_2014['Median_Rent_2014'].str.replace(',', '').str.replace('$','').astype(float)
med_rent_2014.head()

Median_Rent_2014
zip                    
90067            2001.0
90077            2001.0
90094            2001.0
90210            2001.0
90265            2001.0

In [ ]:
med_rent = pd.concat([med_rent_2000,med_rent_2014], axis=1, join='inner')
med_rent['Inflated_Median_Rent_2000'] = 1.3993019 * med_rent['Median_Rent_2014']
med_rent['Rent_Change'] = med_rent['Median_Rent_2014'] - med_rent['Median_Rent_2000']
med_rent = med_rent[['Median_Rent_2000','Inflated_Median_Rent_2000','Median_Rent_2014','Rent_Change']]
med_rent.columns = ['Median Rent 2000','Inflated Median Rent 2000','Median Rent 2014','Rent Change']
med_rent.head()

## Public Transportation

In [29]:
pub_trans_2000 = pd.read_csv('../Resources1/pub_trans_2000.csv')
pub_trans_2000.set_index('zip', inplace=True)
pub_trans_2000 = pub_trans_2000.drop('Unnamed: 0', axis=1)
pub_trans_2000['Take Public Transp % in 2000'] = pub_trans_2000['Take Public Transp % in 2000'].str.rstrip('%').astype('float')/100.0
pub_trans_2000.head()

Take Public Transp % in 2000
zip                                
90017                         0.589
90057                         0.444
95431                         0.430
94102                         0.426
94103                         0.399

In [30]:
pub_trans_2014 = pd.read_csv('../Resources1/pub_trans_2014.csv')
pub_trans_2014.set_index('zip', inplace=True)
pub_trans_2014 = pub_trans_2014.drop('Unnamed: 0', axis=1)
pub_trans_2014['Take Public Transp % in 2014'] = pub_trans_2014['Take Public Transp % in 2014'].str.rstrip('%').astype('float')/100.0
pub_trans_2014.head()

Take Public Transp % in 2014
zip                                
92304                         1.000
94128                         1.000
93634                         0.727
90073                         0.478
94130                         0.476

## Total Population

In [10]:
tot_pop_2000 = pd.read_csv('../Resources1/total_pop_2000.csv')
tot_pop_2000.set_index('zip', inplace=True)
tot_pop_2000 = tot_pop_2000.drop('Unnamed: 0', axis=1)
tot_pop_2000 = tot_pop_2000.astype(float)
tot_pop_2000.head()

Total_Pop_2000
zip                  
90201        105275.0
90650        103211.0
90011        101214.0
92054         98226.0
91331         97300.0

In [11]:
tot_pop_2014 = pd.read_csv('../Resources1/total_pop_2014.csv')
tot_pop_2014.set_index('zip', inplace=True)
tot_pop_2014 = tot_pop_2014.drop('Unnamed: 0', axis=1)
tot_pop_2014 = tot_pop_2014.astype(float)
tot_pop_2014.head()

Total Population % in 2014
zip                              
90650                    106521.0
90011                    102926.0
90201                    102515.0
91331                    102367.0
92335                     99580.0

## White Population

In [17]:
white_pop_2000 = pd.read_csv('../Resources1/white_pop_2000.csv')
white_pop_2000.set_index('zip', inplace=True)
white_pop_2000 = white_pop_2000.drop('Unnamed: 0', axis=1)
white_pop_2000['White Population % in 2000'] = white_pop_2000['White Population % in 2000'].str.rstrip('%').astype('float')/100.0
white_pop_2000.head()

White Population % in 2000
zip                              
90263                         1.0
91743                         1.0
92338                         1.0
95232                         1.0
95735                         1.0

In [19]:
white_pop_2014 = pd.read_csv('../Resources1/white_pop_2014.csv')
white_pop_2014.set_index('zip', inplace=True)
white_pop_2014 = white_pop_2014.drop('Unnamed: 0', axis=1)
white_pop_2014['White Population % in 2014'] = white_pop_2014['White Population % in 2014'].str.rstrip('%').astype('float')/100.0
white_pop_2014.head()

White Population % in 2014
zip                              
91948                         1.0
92060                         1.0
92266                         1.0
92304                         1.0
92332                         1.0

In [20]:
dfs = [avg_edu_2000, avg_edu_2014,med_inc_2000,med_inc_2014,med_rent_2000,med_rent_2014,
pub_trans_2000,pub_trans_2014,tot_pop_2000,tot_pop_2014,white_pop_2000,white_pop_2014]

df_2000=

rename = ['Avg_Edu_Index_2000','Avg_Edu_Index_2014','Index_Change','Median_Income_2000',
          'Median_Income_2014','Median_Rent_2000','Median_Rent_2014',
          'Total_Pop_2000','Total_Pop_2014','White_Pop_2000','White_Pop_2014']

In [21]:
usa_df=pd.concat(dfs, axis=1, join='inner')
usa_df.columns = rename
usa_blank = usa_df
usa_blank['Outcome']=''
usa_blank

Avg_Edu_Index_2000  Avg_Edu_Index_2014  Index_Change  \
zip                                                           
94305               17.76               17.69         -0.07   
94708               17.01               17.12          0.11   
94304               17.00               17.02          0.02   
94707               16.83               17.15          0.32   
94028               16.83               16.76         -0.07   
...                   ...                 ...           ...   
93640                9.09                9.72          0.63   
93261                9.00                9.68          0.68   
93234                8.96                9.49          0.53   
93608                8.90                9.35          0.45   
93249                8.82                9.22          0.40   

       Median_Income_2000  Median_Income_2014  Median_Rent_2000  \
zip                                                               
94305             41313.0             51976.0             843.0   
94708            103791.0            145610.0            1527.0   
94304             77539.0            101932.0            1948.0   
94707            100590.0            136331.0            1076.0   
94028            164479.0            180174.0            2001.0   
...                   ...                 ...               ...   
93640             23815.0             25741.0             448.0   
93261             22011.0             29005.0             400.0   
93234             25521.0             29304.0             364.0   
93608             27604.0             31111.0             418.0   
93249             30625.0             33011.0             438.0   

       Median_Rent_2014  Total_Pop_2000  Total_Pop_2014  White_Pop_2000  \
zip                                                                       
94305            1524.0         13371.0         13538.0           0.606   
94708            1988.0         10730.0         11143.0           0.839   
94304            2001.0          1704.0          3688.0           0.731   
94707            1719.0         11880.0         12402.0           0.832   
94028            2001.0          6595.0          6720.0           0.925   
...                 ...             ...             ...             ...   
93640             663.0          9160.0         12140.0           0.287   
93261             610.0          2945.0          3248.0           0.185   
93234             666.0          6902.0          6896.0           0.206   
93608             853.0          1746.0           725.0           0.395   
93249             619.0          2501.0          2698.0           0.212   

       White_Pop_2014 Outcome  
zip                            
94305           0.590          
94708           0.818          
94304           0.704          
94707           0.812          
94028           0.870          
...               ...     ...  
93640           0.848          
93261           0.617          
93234           0.502          
93608           0.772          
93249           0.933          

[1499 rows x 12 columns]

In [ ]:
usa_blank.to_csv('usa_blank.csv', index = 'zip', header=True)